In [18]:
import pandas as pd
import torch

In [19]:
df = pd.read_csv("./Data/RAW_dataset/KnowlageDataset")

In [ ]:
df

In [21]:
df.drop_duplicates(subset=["question"],inplace=True)
df.drop_duplicates(subset=["response"],inplace=True)


In [22]:
len(df["question"][0]) + len(df["response"][0])

849

In [ ]:
df = df[df["question"].str.len() + df["response"].str.len() <= 512]
df.reset_index(drop=True,inplace=True)
df

In [ ]:
df["combined"] = df["question"] + " " +  df["response"]
df = df.assign(len=df['combined'].str.len()).sort_values('len').drop('len', axis=1)
df.reset_index(inplace=True,drop=True)
df

In [ ]:

df = df[df['response'].map(lambda x: x.isascii())]
df = df[df['question'].map(lambda x: x.isascii())]

df["question"] = df["question"].str.lower()
df["response"] = df["response"].str.lower()
df

In [26]:
combinedText = []
buffer = ""

for i in range(len(df)):
   
    text = f"<QUESTION> {df['question'].iloc[i]} <ANSWER> {df['response'].iloc[i]} <|endoftext|> "
    
   
    if len(buffer.split()) + len(text.split()) <= 350:
        buffer += text
    else:
        
        combinedText.append(buffer)
        buffer = text
        
if buffer:
    combinedText.append(buffer)


In [27]:
len(combinedText[5].split())

341

In [ ]:
from transformers import AutoTokenizer


tokenizerGPT = AutoTokenizer.from_pretrained("gpt2")

customTokenizer = tokenizerGPT.train_new_from_iterator(combinedText, vocab_size=6000,new_special_tokens=["<QUESTION>","<ANSWER>"])

customTokenizer.add_special_tokens({
    'pad_token': '[PAD]'
})
customTokenizer.save_pretrained("./TOKENIZER")

In [ ]:
print(f"Question ID: {customTokenizer.convert_tokens_to_ids('<QUESTION>')}")
print(f"Answer ID: {customTokenizer.convert_tokens_to_ids('<ANSWER>')}")

In [30]:
retrivedTokenizer = AutoTokenizer.from_pretrained("./TOKENIZER/")

In [ ]:
customTokenizer("Ask")

In [32]:
outputTokenizer = retrivedTokenizer(
    combinedText,
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors="pt"
)

In [33]:
IDs = outputTokenizer["input_ids"]
masks = outputTokenizer["attention_mask"]

In [ ]:
torch.save(IDs,"./Data/preproccessedData.pt")
torch.save(masks,"./Data/preproccessedDataMASKS.pt")

tokenCOUNT = outputTokenizer["attention_mask"].sum().item()
tokenCOUNT